In [1]:
import sys
import os
import inspect

currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir) 


In [2]:
from data.Clustering import UnsupervisedLearning

In [3]:
import numpy as np
import pandas as pd
from stable_baselines import PPO2
from stable_baselines.common.vec_env import DummyVecEnv
from env.EnvStock_trade import StockEnvTrade
import tensorflow as tf
import requests
import yfinance as yf
from utils import pbt
#from tensorboard.plugins.hparams import api as hp
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

C:\Users\kaanb\Anaconda3\envs\agents\lib\site-packages\stable_baselines\__init__.py:33: UserWarning: stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.
  "stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html

In [4]:
def get_price(ticker,start, end):
    stock = yf.Ticker(ticker)
    
    df = stock.history(start=start, end = end).reset_index()

        
    print(df)
    df['Adj Close'] = df['Close']
    df['ticker'] = ticker
    return df

In [6]:
df = pd.read_csv('../data/all_stocks.csv')

In [7]:
df.head()

,Unnamed: 0,Symbol,Name,Sector
0,0.0,MMM,3M Company,Industrials
1,1.0,AOS,A.O. Smith Corp,Industrials
2,2.0,ABT,Abbott Laboratories,Health Care
3,3.0,ABBV,AbbVie Inc.,Health Care
4,4.0,ACN,Accenture plc,Information Technology


In [8]:
symbols = df.Symbol.unique()

In [9]:
stock = yf.Ticker('TSLA')
df = stock.history(start='2010-01-01', end = '2011-01-01')


In [10]:
tsla = get_price('TSLA','2007-01-01','2011-01-01')

          Date   Open   High    Low  Close    Volume  Dividends  Stock Splits
0   2010-06-29  3.800  5.000  3.508  4.778  93831500          0             0
1   2010-06-30  5.158  6.084  4.660  4.766  85935500          0             0
2   2010-07-01  5.000  5.184  4.054  4.392  41094000          0             0
3   2010-07-02  4.600  4.620  3.742  3.840  25699000          0             0
4   2010-07-06  4.000  4.000  3.166  3.222  34334500          0             0
..         ...    ...    ...    ...    ...       ...        ...           ...
125 2010-12-27  5.604  5.716  5.012  5.110  46509500          0             0
126 2010-12-28  5.170  5.350  5.000  5.282  20281500          0             0
127 2010-12-29  5.406  5.602  5.300  5.546  16596000          0             0
128 2010-12-30  5.540  5.580  5.276  5.300  10205500          0             0
129 2010-12-31  5.314  5.450  5.300  5.326   7089500          0             0

[130 rows x 8 columns]


In [11]:
temp1 = ['CAT', 'AES', 'BA', 'ADI', 'BWA', 'A', 'BKR', 'COF', 'CBRE', 'MMM', 'AIG', 'AKAM', 'CCL', 'BAC', 'AMD', 'BSX', 'AMAT', 'BK', 'AAPL', 'CF', 'ATVI', 'AMZN', 'CERN', 'ABT', 'MO', 'BMY', 'COG', 'BBWI', 'T', 'AXP', 'BAX', 'ACN', 'ADP', 'AMGN', 'CDNS', 'CNC', 'ALK', 'AJG', 'GOOG', 'GOOGL', 'AAP', 'AIZ', 'CBOE', 'ATO']
temp2 = ['AMD', 'BSX', 'ATVI', 'BBY', 'AES', 'BA', 'CNC', 'CAH', 'CHRW', 'BR', 'AMAT', 'CAT', 'BKR', 'A', 'APA', 'BAC', 'BK', 'AFL', 'BAX', 'ADBE', 'COG', 'GOOG', 'GOOGL', 'CCL', 'ABC', 'APTV', 'AAPL', 'AXP', 'CERN', 'AVGO', 'BKNG', 'BMY', 'AEE', 'ALGN', 'BXP', 'AVB', 'AIG', 'CF', 'COF', 'ACN', 'BBWI', 'T', 'ABT', 'MO', 'AAL', 'AMGN']

list(set(temp1) - set(temp2))

['BWA',
 'CDNS',
 'ALK',
 'CBOE',
 'ADI',
 'AMZN',
 'ADP',
 'AKAM',
 'AJG',
 'MMM',
 'AAP',
 'ATO',
 'CBRE',
 'AIZ']

In [12]:
tsla = get_price('TSLA','2011-01-01','2012-01-01')
aapl = get_price('AAPL', '2011-01-01','2012-01-01')

          Date   Open   High    Low  Close    Volume  Dividends  Stock Splits
0   2010-12-31  5.314  5.450  5.300  5.326   7089500          0             0
1   2011-01-03  5.368  5.400  5.180  5.324   6415000          0             0
2   2011-01-04  5.332  5.390  5.204  5.334   5937000          0             0
3   2011-01-05  5.296  5.380  5.238  5.366   7233500          0             0
4   2011-01-06  5.366  5.600  5.362  5.576  10306000          0             0
..         ...    ...    ...    ...    ...       ...        ...           ...
248 2011-12-23  5.600  5.600  5.504  5.580   2957000          0             0
249 2011-12-27  5.532  5.754  5.528  5.714   3887500          0             0
250 2011-12-28  5.798  5.848  5.608  5.702   2876000          0             0
251 2011-12-29  5.718  5.868  5.710  5.746   2441000          0             0
252 2011-12-30  5.698  5.796  5.650  5.712   1699000          0             0

[253 rows x 8 columns]
          Date       Open       High    

In [13]:
df = pd.concat([tsla, aapl])

In [14]:
df

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Adj Close,ticker
0,2010-12-31,5.314000,5.450000,5.300000,5.326000,7089500,0,0,5.326000,TSLA
1,2011-01-03,5.368000,5.400000,5.180000,5.324000,6415000,0,0,5.324000,TSLA
2,2011-01-04,5.332000,5.390000,5.204000,5.334000,5937000,0,0,5.334000,TSLA
3,2011-01-05,5.296000,5.380000,5.238000,5.366000,7233500,0,0,5.366000,TSLA
4,2011-01-06,5.366000,5.600000,5.362000,5.576000,10306000,0,0,5.576000,TSLA
...,...,...,...,...,...,...,...,...,...,...
248,2011-12-23,12.256442,12.376035,12.250309,12.368062,269399200,0,0,12.368062,AAPL
249,2011-12-27,12.361008,12.544691,12.358554,12.466188,265076000,0,0,12.466188,AAPL
250,2011-12-28,12.477230,12.518933,12.307039,12.346904,228662000,0,0,12.346904,AAPL
251,2011-12-29,12.370210,12.439206,12.281589,12.422953,215978000,0,0,12.422953,AAPL


In [15]:
data = []
headers = []
full_stock_df = []
for tick in df.ticker.unique():
    index= df.Date.unique()
    stock = df[df.ticker == tick]
    data.append(stock['Close'])
    headers.append(tick)
    full_stock_df.append(stock.set_index('Date'))
    

In [16]:
headers

['TSLA', 'AAPL']

In [17]:
from datetime import date, timedelta

start_date = date(2010, 1, 1)
end_date = date(2020, 1, 1)
delta = timedelta(days=365)
while start_date <= end_date:
    print(start_date.strftime("%Y-%m-%d"))
    start_date += delta

2010-01-01
2011-01-01
2012-01-01
2012-12-31
2013-12-31
2014-12-31
2015-12-31
2016-12-30
2017-12-30
2018-12-30
2019-12-30


In [18]:
stock_prices = pd.DataFrame()
for i in range(len(data)):
    stock_prices[headers[i]] = data[i]
stock_prices.set_index(index, inplace=True)
stock_prices.head()

,TSLA,AAPL
2010-12-31,5.326,9.891260
2011-01-03,5.324,10.106222
2011-01-04,5.334,10.158965
2011-01-05,5.366,10.242066
2011-01-06,5.576,10.233788


In [33]:
from data.data_generator import *

In [19]:
UL = UnsupervisedLearning(df)

In [23]:
UL.dataframe

,0,1,2,3,4,5,6,7,8,9,...,243,244,245,246,247,248,249,250,251,252
TSLA,0.366133,0.365370,0.369184,0.381388,0.461480,0.488940,0.504958,0.391304,0.391304,0.334859,...,0.470633,0.451564,0.463005,0.437834,0.453089,0.463005,0.514111,0.509535,0.526316,0.513349
AAPL,0.067715,0.133277,0.149365,0.174710,0.172185,0.194538,0.253742,0.246166,0.272167,0.283951,...,0.614478,0.625608,0.754116,0.758792,0.778432,0.823139,0.853068,0.816685,0.839880,0.838758


In [22]:
cluster = UL.kmeans_cluster_stocks(2)

In [23]:
cluster

,label,distance
index,,
TSLA,0,4.438186
AAPL,1,4.438186


In [24]:
from data.data_generator import pick_from_kmeans_cluster 

In [31]:
stk = pick_from_kmeans_cluster(cluster, df)

                 0
TSLA  6.455593e+06
AAPL  4.911180e+08


In [32]:
stk

[[], ['AAPL']]